<a href="https://colab.research.google.com/github/arefeh-rj/reasoning/blob/main/new_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q transformers accelerate bitsandbytes peft pandas tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import time
import torch
import gc
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

def evaluate_models():
    models_to_evaluate = [
        "arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit",
        "arefehRajabian/qwenMath",
        "arefehRajabian/gemma-3-4b-it-Persian",
        "arefehRajabian/Qwen2.5-7B-Instruct",
        "arefehRajabian/Qwen2.5-7B-PersianMath-LoRA"
    ]

    evaluation_dataset = [
        {"question": "حل معادله زیر: ۲x + ۳ = ۷", "expected_answer": "x = 2"},
        {"question": "قیمت یک کتاب ۵۰۰۰ تومان است. اگر ۱۰ درصد تخفیف داشته باشد، قیمت جدید چقدر است؟", "expected_answer": "4500"},
        {"question": "حاصل ۳ * (۴ + ۵) چیست؟", "expected_answer": "27"}
    ]

    results = []

    for model_name in models_to_evaluate:
        print(f"\n--- Starting Evaluation: {model_name} ---")

        try:
            # 1. Clear GPU memory before loading new model
            if 'pipe' in locals(): del pipe
            torch.cuda.empty_cache()
            gc.collect()

            # 2. Load Model & Tokenizer
            # Using 4-bit to ensure the 7B models fit in Colab memory
            pipe = pipeline(
                "text-generation",
                model=model_name,
                device_map="auto",
                model_kwargs={
                    "load_in_4bit": True,
                    "bnb_4bit_compute_dtype": torch.bfloat16,
                    "trust_remote_code": True
                }
            )

            for item in evaluation_dataset:
                question = item["question"]
                expected_answer = item["expected_answer"]

                start_time = time.time()
                # temperature 0.1 for deterministic/math-focused output
                output = pipe(question, max_new_tokens=100, temperature=0.1, do_sample=False)
                end_time = time.time()

                generated_text = output[0]['generated_text']
                # Basic Persian/Arabic number normalization could be added here
                is_correct = expected_answer in generated_text

                results.append({
                    "Model": model_name,
                    "Question": question,
                    "Generated Answer": generated_text.replace(question, "").strip()[:100], # Clean output
                    "Correct": is_correct,
                    "Inference Time (s)": round(end_time - start_time, 2)
                })
                print(f"  Done: {question[:20]}...")

        except Exception as e:
            print(f"FAILED to load {model_name}: {e}")
            results.append({"Model": model_name, "Question": "ALL", "Generated Answer": "ERROR", "Correct": False, "Inference Time (s)": 0})

    # --- Data Processing ---
    results_df = pd.DataFrame(results)

    # Calculate Summary
    summary = results_df.groupby('Model').agg(
        Correct_Count=('Correct', 'sum'),
        Avg_Time=('Inference Time (s)', 'mean')
    ).reset_index()
    summary['Accuracy (%)'] = (summary['Correct_Count'] / len(evaluation_dataset)) * 100

    print("\n" + "="*50)
    print("FINAL COMPARISON")
    print("="*50)
    print(summary.to_markdown(index=False))

# Run it
evaluate_models()


--- Starting Evaluation: arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json: 0.00B [00:00, ?B/s]

FAILED to load arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit: transformers.models.auto.configuration_auto.AutoConfig.from_pretrained() got multiple values for keyword argument 'trust_remote_code'

--- Starting Evaluation: arefehRajabian/qwenMath ---


adapter_config.json:   0%|          | 0.00/859 [00:00<?, ?B/s]

FAILED to load arefehRajabian/qwenMath: transformers.models.auto.configuration_auto.AutoConfig.from_pretrained() got multiple values for keyword argument 'trust_remote_code'

--- Starting Evaluation: arefehRajabian/gemma-3-4b-it-Persian ---
FAILED to load arefehRajabian/gemma-3-4b-it-Persian: transformers.models.auto.configuration_auto.AutoConfig.from_pretrained() got multiple values for keyword argument 'trust_remote_code'

--- Starting Evaluation: arefehRajabian/Qwen2.5-7B-Instruct ---


adapter_config.json: 0.00B [00:00, ?B/s]

FAILED to load arefehRajabian/Qwen2.5-7B-Instruct: transformers.models.auto.configuration_auto.AutoConfig.from_pretrained() got multiple values for keyword argument 'trust_remote_code'

--- Starting Evaluation: arefehRajabian/Qwen2.5-7B-PersianMath-LoRA ---


adapter_config.json: 0.00B [00:00, ?B/s]

FAILED to load arefehRajabian/Qwen2.5-7B-PersianMath-LoRA: transformers.models.auto.configuration_auto.AutoConfig.from_pretrained() got multiple values for keyword argument 'trust_remote_code'

FINAL COMPARISON
| Model                                      |   Correct_Count |   Avg_Time |   Accuracy (%) |
|:-------------------------------------------|----------------:|-----------:|---------------:|
| arefehRajabian/Qwen2.5-7B-Instruct         |               0 |          0 |              0 |
| arefehRajabian/Qwen2.5-7B-PersianMath-LoRA |               0 |          0 |              0 |
| arefehRajabian/gemma-3-4b-it-Persian       |               0 |          0 |              0 |
| arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit |               0 |          0 |              0 |
| arefehRajabian/qwenMath                    |               0 |          0 |              0 |


In [3]:
import pandas as pd
import time
import torch
import gc
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

def evaluate_models():
    models_to_evaluate = [
        "arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit",
        "arefehRajabian/qwenMath",
        "arefehRajabian/gemma-3-4b-it-Persian",
        "arefehRajabian/Qwen2.5-7B-Instruct",
        "arefehRajabian/Qwen2.5-7B-PersianMath-LoRA"
    ]

    evaluation_dataset = [
        {"question": "حل معادله زیر: ۲x + ۳ = ۷", "expected_answer": "x = 2"},
        {"question": "قیمت یک کتاب ۵۰۰۰ تومان است. اگر ۱۰ درصد تخفیف داشته باشد، قیمت جدید چقدر است؟", "expected_answer": "4500"},
        {"question": "حاصل ۳ * (۴ + ۵) چیست؟", "expected_answer": "27"}
    ]

    # Configuration for 4-bit loading (to save memory)
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    results = []

    for model_name in models_to_evaluate:
        print(f"\n--- Loading Model: {model_name} ---")
        try:
            # Cleanup GPU
            torch.cuda.empty_cache()
            gc.collect()

            # Load Tokenizer
            tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
            if tokenizer.pad_token is None:
                tokenizer.pad_token = tokenizer.eos_token

            # Load Model manually to avoid the 'multiple values' error
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=bnb_config,
                device_map="auto",
                trust_remote_code=True
            )

            # Create pipeline with pre-loaded model/tokenizer
            pipe = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer
            )

            for item in evaluation_dataset:
                question = item["question"]
                expected_answer = item["expected_answer"]

                start_time = time.time()
                # Use a specific prompt format if needed, here we keep it simple
                output = pipe(question, max_new_tokens=64, temperature=0.1, do_sample=False)
                end_time = time.time()

                full_text = output[0]['generated_text']
                # Remove the prompt from the answer
                generated_answer = full_text.replace(question, "").strip()

                results.append({
                    "Model": model_name,
                    "Question": question,
                    "Generated Answer": generated_answer[:100], # truncate for table
                    "Correct": expected_answer in generated_answer,
                    "Inference Time (s)": round(end_time - start_time, 2)
                })
                print(f"  - Processed: {question[:15]}...")

            # Final Cleanup for this model
            del pipe
            del model
            del tokenizer
            gc.collect()
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"FAILED {model_name}: {str(e)}")

    # Create Table
    df = pd.DataFrame(results)
    if not df.empty:
        summary = df.groupby('Model').agg({'Correct': 'sum', 'Inference Time (s)': 'mean'}).reset_index()
        print("\n", summary.to_markdown(index=False))
    else:
        print("No results to display.")

evaluate_models()


--- Loading Model: arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit ---


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

chat_template.jinja: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/323M [00:00<?, ?B/s]

Device set to use cuda:0


  - Processed: حل معادله زیر: ...
  - Processed: قیمت یک کتاب ۵۰...
  - Processed: حاصل ۳ * (۴ + ۵...

 | Model                                      |   Correct |   Inference Time (s) |
|:-------------------------------------------|----------:|---------------------:|
| arefehRajabian/Qwen2.5-7B-Instruct         |         1 |              8.88    |
| arefehRajabian/Qwen2.5-7B-PersianMath-LoRA |         0 |              8.57667 |
| arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit |         0 |             10.5867  |
| arefehRajabian/qwenMath                    |         0 |              6.85333 |


In [4]:
import pandas as pd
import time
import torch
import gc
import re
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from IPython.display import display, HTML

# 1. Normalization helper to handle Persian/English number mismatches
def normalize_text(text):
    if not isinstance(text, str): return ""
    # Convert Arabic/Persian digits to English
    p_digits = "۰۱۲۳۴۵۶۷۸۹"
    a_digits = "٠١٢٣٤٥٦٧٨٩"
    for i in range(10):
        text = text.replace(p_digits[i], str(i)).replace(a_digits[i], str(i))
    # Basic cleanup
    text = text.replace("ی", "ی").replace("ک", "ک") # Standardize Y/K
    return text.strip()

def evaluate_models():
    models_to_evaluate = [
        "arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit",
        "arefehRajabian/qwenMath",
        "arefehRajabian/gemma-3-4b-it-Persian",
        "arefehRajabian/Qwen2.5-7B-Instruct",
        "arefehRajabian/Qwen2.5-7B-PersianMath-LoRA"
    ]

    evaluation_dataset = [
        {"question": "حل معادله زیر: ۲x + ۳ = ۷", "expected_answer": "2"},
        {"question": "قیمت یک کتاب ۵۰۰۰ تومان است. اگر ۱۰ درصد تخفیف داشته باشد، قیمت جدید چقدر است؟", "expected_answer": "4500"},
        {"question": "حاصل ۳ * (۴ + ۵) چیست؟", "expected_answer": "27"}
    ]

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    )

    all_results = []

    for model_name in models_to_evaluate:
        print(f"--- Processing: {model_name} ---")
        try:
            # Cleanup GPU memory
            if 'model' in locals(): del model, tokenizer, pipe
            torch.cuda.empty_cache()
            gc.collect()

            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                quantization_config=bnb_config,
                device_map="auto",
                trust_remote_code=True
            )

            pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

            for item in evaluation_dataset:
                start_time = time.time()
                # temperature 0.1 for precision
                output = pipe(item["question"], max_new_tokens=100, do_sample=False, temperature=0.1)
                latency = time.time() - start_time

                raw_ans = output[0]['generated_text'].replace(item["question"], "").strip()

                # Check correctness using normalized strings
                is_correct = normalize_text(item["expected_answer"]) in normalize_text(raw_ans)

                all_results.append({
                    "Model": model_name.split('/')[-1], # Short name
                    "Question": item["question"],
                    "Answer": raw_ans,
                    "Correct": "✅" if is_correct else "❌",
                    "Time (s)": round(latency, 2)
                })

        except Exception as e:
            print(f"Skipping {model_name} due to error: {e}")

    # Create and Display the Comparison Table
    df = pd.DataFrame(all_results)

    # Pivoting so questions are columns and models are rows
    comparison_pivot = df.pivot(index='Model', columns='Question', values='Answer')

    print("\n### Model Answer Comparison ###")
    display(HTML(df.to_html())) # Displays a nice table in Colab cells

    return df

# Execute
results_df = evaluate_models()

--- Processing: arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit ---


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Device set to use cuda:0


--- Processing: arefehRajabian/qwenMath ---


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading adapter weights from arefehRajabian/qwenMath led to unexpected keys not found in the model: base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight, base_model.model.model.layers.0.mlp.down_proj.lora_B.default.weight, base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight, base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight, base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight, base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight, base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight, base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight, base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight, base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight, base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight, base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight, base_model.model.model.layers.0.self_attn.v_proj.lora_A.de

...

--- Processing: arefehRajabian/Qwen2.5-7B-PersianMath-LoRA ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0



### Model Answer Comparison ###


,Model,Question,Answer,Correct,Time (s)
0,qwen2.5-3b-unsloth-bnb-4bit,حل معادله زیر: ۲x + ۳ = ۷,- x\nاگر می‌خواهید حل معادله زیر را که ۲x + ۳ = ۷ - x باشد، می‌توانید از روش‌های زیر استفاده کنید:\n\n1. ابتدا هر طرف معادله را به صورت هم‌ریخته کنید. در اینجا، هر طرف,✅,16.51
1,qwen2.5-3b-unsloth-bnb-4bit,قیمت یک کتاب ۵۰۰۰ تومان است. اگر ۱۰ درصد تخفیف داشته باشد، قیمت جدید چقدر است؟,اگر ۱۰ درصد تخفیف داشته باشد، قیمت جدید کتاب برابر است با ۵۰۰۰ تومان - ۱۰ درصد از ۵۰۰۰ تومان.\n\n۱۰ درصد از ۵۰۰۰ تومان به صورت زیر محاسبه می,❌,16.53
2,qwen2.5-3b-unsloth-bnb-4bit,حاصل ۳ * (۴ + ۵) چیست؟,اگر ۳ * (۴ + ۵) را حل کنید، ابتدا باید ۴ و ۵ را با هم جمع کنید:\n\n4 + 5 = 9\n\nسپس، ۳ * ۹ را حل کنید:\n\n3 * 9 = 27\n\nپس، ۳ * (۴ + ۵) = �,✅,16.15
3,qwenMath,حل معادله زیر: ۲x + ۳ = ۷,برای حل معادله ۲x + ۳ = ۷، ابتدا باید مقدار x را پیدا کنیم. برای این منظور، مراحل زیر را دنبال می‌کنیم:\n\n1. ابتدا مقدار ثابت‌ها را به طرف دیگر معادله منتقل می‌کنیم:,✅,9.05
4,qwenMath,قیمت یک کتاب ۵۰۰۰ تومان است. اگر ۱۰ درصد تخفیف داشته باشد، قیمت جدید چقدر است؟,برای محاسبه قیمت جدید کتاب پس از اعمال تخفیف، مراحل زیر را طی می‌کنیم:\n\n1. ابتدا مقدار تخفیف را محاسبه می‌کنیم:\n ۱۰ درصد از ۵۰۰۰ تومان = (۱۰/۱۰,❌,9.14
5,qwenMath,حاصل ۳ * (۴ + ۵) چیست؟,حاصل عبارت ۳ * (۴ + ۵) را می‌توان به این شکل محاسبه کرد:\n\n۱. ابتدا جمع داخل پرانتز را انجام می‌دهیم:\n ۴ + ۵ = ۹\n\n۲. سپس نتیجه را با عدد خارجی ضرب می‌کن,❌,9.07
6,Qwen2.5-7B-Instruct,حل معادله زیر: ۲x + ۳ = ۷,#### 2\n\n#### 2\n#### 2\n\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2\n#### 2,✅,13.07
7,Qwen2.5-7B-Instruct,قیمت یک کتاب ۵۰۰۰ تومان است. اگر ۱۰ درصد تخفیف داشته باشد، قیمت جدید چقدر است؟,#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n#### 4500\n\n####,✅,13.09
8,Qwen2.5-7B-Instruct,حاصل ۳ * (۴ + ۵) چیست؟,ابتدا داخل پرانتز جمع کنید: ۴ + ۵ = <<4+5=9>> ۹\nسپس آن را در ۳ ضرب کنید: ۳ * ۹ = <<3*9=27>> ۲۷\nبنابراین، ۳ * (۴ + ۵) برابر است با ۲۷.,✅,13.15
9,Qwen2.5-7B-PersianMath-LoRA,حل معادله زیر: ۲x + ۳ = ۷,برای حل این معادله، باید مقدار \( x \) را پیدا کنیم. برای این کار، مراحل زیر را دنبال می‌کنیم:\n\n1. **انتقال عبارت ثابت به سمت راست معادله:** ابتدا عبارت ثابت \( 3 \) را از هر دو طرف معادله ح,❌,12.98


In [9]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
test = "gsm8k_test_translated_all.xlsx"
train = "gsm8k_train_translated_all.xlsx"

# Load the latest version
test_dataset = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "arefehrajabian/persian-gsm",
  test,

)

train_dataset = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "arefehrajabian/persian-gsm",
  train,

)
train_dataset.head()

/tmp/ipython-input-1223728155.py:9: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  test_dataset = kagglehub.load_dataset(


100%|██████████| 298k/298k [00:00<00:00, 1.24MB/s]
/tmp/ipython-input-1223728155.py:16: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  train_dataset = kagglehub.load_dataset(


Using Colab cache for faster access to the 'persian-gsm' dataset.


,question,answer,translated_question,steps,translated_steps
0,Natalia sold clips to 48 of her friends in Apr...,72,ناتالیا در ماه آوریل گیره به ۴۸ نفر از دوستانش...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...,ناتالیا در ماه می 48/2 = <<48/2=24>>24 گیره فر...
1,Weng earns $12 an hour for babysitting. Yester...,10,ونگ در ساعت ۱۲ دلار برای نگهداری از کودک دستمز...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...,ونگ 12/60 = $<<12/60=0.2>>0.2 در هر دقیقه درآم...
2,Betty is saving money for a new wallet which c...,5,بتي در حال پس‌انداز پول برای یک کیف پول جدید ا...,"In the beginning, Betty has only 100 / 2 = $<<...",در ابتدا، بتی فقط 100 / 2 = $<<100/2=50>>50 دل...
3,"Julie is reading a 120-page book. Yesterday, s...",42,جولی در حال خواندن کتابی ۱۲۰ صفحه‌ای است. او د...,Maila read 12 x 2 = <<12*2=24>>24 pages today....,میلا امروز 12 x 2 = <<12*2=24>>24 صفحه خواند.\...
4,James writes a 3-page letter to 2 different fr...,624,جیمز هر هفته دو بار نامه‌های ۳ صفحه‌ای به ۲ دو...,He writes each friend 3*2=<<3*2=6>>6 pages a w...,او برای هر دوست ۳*۲=<<3*2=6>> ۶ صفحه در هفته م...


In [12]:
import pandas as pd
import re
import torch
import gc
import time
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from IPython.display import display, HTML

# 1. SETUP MODEL (Re-initializing the pipe)
target_model = "arefehRajabian/Qwen2.5-7B-PersianMath-LoRA"

print(f"--- Loading Model: {target_model} ---")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(target_model)
model = AutoModelForCausalLM.from_pretrained(
    target_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Initialize the missing 'pipe'
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 2. CONFIGURATION & DATASET
# Using the translated_question column from your Kaggle dataset
eval_df = test_dataset.head(10)
system_prompt = "شما یک متخصص ریاضی هستید. سوال را مرحله به مرحله حل کنید و در انتها پاسخ نهایی را بنویسید."

model_results = []

print(f"--- Evaluating {len(eval_df)} GSM8K Questions ---")

for index, row in eval_df.iterrows():
    question = row['translated_question']
    ground_truth = str(row['answer'])

    # Prompt structure
    prompt = f"{system_prompt}\n\nسوال: {question}\nپاسخ:"

    # Generate
    output = pipe(prompt, max_new_tokens=350, temperature=0.1, do_sample=False)
    generated_text = output[0]['generated_text']

    # Clean the output to get just the model's response
    answer_only = generated_text.replace(prompt, "").strip()

    model_results.append({
        "Index": index,
        "Question": question,
        "Model's Logic": answer_only,
        "Target Answer": ground_truth
    })
    print(f"Done processing question {index+1}")

# 3. ANALYSIS & DISPLAY
results_df = pd.DataFrame(model_results)

# Number normalization function for accuracy check
def check_accuracy(row):
    def to_eng(text):
        p_digits = "۰۱۲۳۴۵۶۷۸۹"
        for i, d in enumerate(p_digits):
            text = text.replace(d, str(i))
        return text

    # We extract all numbers from the model output and see if the target is there
    model_nums = re.findall(r'\d+', to_eng(row["Model's Logic"]))
    target_num = to_eng(row["Target Answer"])
    return target_num in model_nums

results_df['Correct'] = results_df.apply(check_accuracy, axis=1)

# Display table
display(HTML("<h2>GSM8K Persian Evaluation Results</h2>"))
display(results_df.style.set_properties(**{
    'text-align': 'right',
    'direction': 'rtl',
    'white-space': 'pre-wrap'
}))

accuracy = results_df['Correct'].mean() * 100
print(f"\n⭐ Sample Accuracy: {accuracy}%")

--- Loading Model: arefehRajabian/Qwen2.5-7B-PersianMath-LoRA ---


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


--- Evaluating 10 GSM8K Questions ---
Done processing question 1
Done processing question 2
Done processing question 3
Done processing question 4
Done processing question 5
Done processing question 6
Done processing question 7
Done processing question 8
Done processing question 9
Done processing question 10


,Index,Question,Model's Logic,Target Answer,Correct
0,0,مرغ‌های جانت هر روز ۱۶ تخم می‌گذارند. او هر صبح سه تخم برای صبحانه می‌خورد و هر روز چهار تخم را برای پختن مافین برای دوستانش استفاده می‌کند. او باقی‌مانده تخم‌ها را به قیمت ۲ دلار برای هر تخم مرغ تازه در بازار کشاورزان به صورت روزانه می‌فروشد. او هر روز چقدر دلار در بازار کشاورزان به دست می‌آورد؟,خب، برای پاسخ دادن به این سوال، باید مراحل زیر را طی کنیم: **1. محاسبه تعداد تخم‌های باقی‌مانده:** * جانت هر روز ۱۶ تخم می‌گذارد. * او سه تخم برای صبحانه می‌خورد. * او چهار تخم برای پختن مافین می‌کند. بنابراین، تعداد تخم‌هایی که او می‌خورد و مافین می‌پخت، به این صورت محاسبه می‌شود: $$3 + 4 = 7$$ حالا، تعداد تخم‌های باقی‌مانده به این صورت محاسبه می‌شود: $$16 - 7 = 9$$ بنابراین، جانت ۹ تخم باقی‌مانده دارد. **2. محاسبه درآمد از فروش تخم‌ها:** * جانت هر تخم را به قیمت ۲ دلار در بازار کشاورزان فروش می‌دهد. * او ۹ تخم باقی‌مانده دارد. بنابراین، درآمدش به این صورت محاسبه می‌شود: $$9 \times 2 = 18$$ **پاس,18,True
1,1,یک ردای عبا ۲ گُل (بُلت) الیاف آبی و نصف آن الیاف سفید نیاز دارد. در مجموع چند گُل الیاف لازم است؟,برای حل این مسئله، باید مقدار گُل‌های الیاف آبی و سفید را جداگانه محاسبه کرده و سپس آن‌ها را با هم جمع کنیم. 1. **محاسبه گُل‌های الیاف آبی:** * می‌دانیم که یک ردای عبا 2 گُل الیاف آبی نیاز دارد. * بنابراین، گُل‌های الیاف آبی = 2 گُل 2. **محاسبه گُل‌های الیاف سفید:** * می‌دانیم که یک ردای عبا نصف گُل الیاف آبی نیاز دارد. * یعنی گُل‌های الیاف سفید = 1/2 × گُل‌های الیاف آبی * گُل‌های الیاف سفید = 1/2 × 2 = 1 گُل 3. **جمع گُل‌های الیاف آبی و سفید:** * گُل‌های کلی = گُل‌های الیاف آبی + گُل‌های الیاف سفید *,3,True
2,2,"جاش تصمیم می‌گیرد یک خانه را بازسازی و بفروشد. او یک خانه به قیمت ۸۰,۰۰۰ دلار می‌خرد و سپس ۵۰,۰۰۰ دلار برای تعمیرات آن هزینه می‌کند. این کار ارزش خانه را ۱۵۰ درصد افزایش می‌دهد. او چقدر سود کرد؟","خب، برای پیدا کردن سود جاش، باید مراحل زیر را دنبال کنیم: **1. محاسبه ارزش خانه بعد از تعمیرات:** * قیمت اولیه خانه: $80,000 * هزینه تعمیرات: $50,000 **ارزش خانه بعد از تعمیرات = قیمت اولیه + هزینه تعمیرات** $80,000 + $50,000 = $130,000 **2. محاسبه ارزش خانه بعد از افزایش 150%:** * افزایش 150% از $130,000 چقدر است؟ **افزایش = (150/100) × $130,000 = 1.5 × $130,000 = $195,000** * ارزش خانه بعد از افزایش = ارزش خانه قبل از افزایش + افزایش $130,000 + $195,000 = $325,000 **3. محاسبه سود:** * سود = ارزش خانه بعد از فروش - هز",70000,False
3,3,جیمز تصمیم می‌گیرد ۳ دوی سرعت را ۳ بار در هفته بدود. او در هر دوی سرعت ۶۰ متر می‌دود. او در مجموع چند متر در هفته می‌دود؟,خب، برای پاسخ دادن به این سوال، مراحل زیر را دنبال می‌کنیم: **مرحله 1: تعیین مسافت در هر دوی سرعت** جیمز در هر دوی سرعت ۶۰ متر می‌دود. بنابراین، مسافت در هر دوی سرعت ۶۰ متر است. **مرحله 2: تعیین تعداد دوی سرعت در هفته** جیمز ۳ دوی سرعت را ۳ بار در هفته بدود. بنابراین، تعداد کل دوی سرعت در هفته برابر است با: $3 \text{ دوی سرعت} \times 3 = 9 \text{ دوی سرعت}$ **مرحله 3: محاسبه مجموع مسافت در هفته** حالا که مسافت در هر دوی سرعت (۶۰ متر) و تعداد کل دوی سرعت در هفته (۹ دوی سرعت) را داریم، می‌توانیم مجموع مسافت در هفته را محاسبه کنیم: $60 \text{ متر} \times 9 = 540 \text{ متر}$ **پاسخ نهایی:**,540,True
4,4,هر روز، وندی به هر یک از مرغ‌هایش سه پیمانه خوراک مخلوط مرغ می‌دهد که شامل دانه‌ها، کرم خاکی و سبزیجات است تا به حفظ سلامتی آن‌ها کمک کند. او خوراک مرغ‌ها را در سه وعده جداگانه می‌دهد. صبح، ۱۵ پیمانه خوراک به گله مرغ‌هایش می‌دهد. بعدازظهر، ۲۵ پیمانه خوراک دیگر به مرغ‌هایش می‌دهد. اگر تعداد مرغ‌های وندی ۲۰ عدد باشد، او برای وعده نهایی روز به چند پیمانه خوراک نیاز دارد؟,برای حل این مسئله، باید مراحل زیر را دنبال کنیم: **1. محاسبه کل مقدار خوراکی که وندی به مرغ‌ها داده است:** * صبح: ۱۵ پیمانه * بعدازظهر: ۲۵ پیمانه بنابراین، کل مقدار خوراکی که وندی به مرغ‌ها داده است، برابر است با: $$ 15 + 25 = 40 \text{ پیمانه} $$ **2. محاسبه مقدار خوراکی مورد نیاز برای هر مرغ:** * وندی ۲۰ مرغ دارد. * کل مقدار خوراکی که به مرغ‌ها داده شده است، ۴۰ پیمانه است. بنابراین، مقدار خوراکی مورد نیاز برای هر مرغ، برابر است با: $$ \frac{40}{20} = 2 \text{ پیمانه} $$ **3. محاسبه مقدار خوراکی مورد نیاز برای وعده نهایی روز:** * وندی ۲۰ مرغ دارد. * مقدار خوراکی م


⭐ Sample Accuracy: 40.0%


In [16]:
import torch
import gc

# 1. Delete the previous model and pipeline
if 'pipe' in locals(): del pipe
if 'model' in locals(): del model
if 'tokenizer' in locals(): del tokenizer

# 2. Force garbage collection
gc.collect()

# 3. Clear the VRAM cache
torch.cuda.empty_cache()

print("GPU Memory Cleared! You can now load the next model.")

GPU Memory Cleared! You can now load the next model.


In [17]:
import pandas as pd
import re
import torch
import gc
import time
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from IPython.display import display, HTML

# 1. SETUP MODEL (Re-initializing the pipe)
target_model = "arefehRajabian/qwenMath"

print(f"--- Loading Model: {target_model} ---")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(target_model)
model = AutoModelForCausalLM.from_pretrained(
    target_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Initialize the missing 'pipe'
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 2. CONFIGURATION & DATASET
# Using the translated_question column from your Kaggle dataset
eval_df = test_dataset.head(10)
system_prompt = "شما یک متخصص ریاضی هستید. سوال را مرحله به مرحله حل کنید و در انتها پاسخ نهایی را بنویسید."

model_results = []

print(f"--- Evaluating {len(eval_df)} GSM8K Questions ---")

for index, row in eval_df.iterrows():
    question = row['translated_question']
    ground_truth = str(row['answer'])

    # Prompt structure
    prompt = f"{system_prompt}\n\nسوال: {question}\nپاسخ:"

    # Generate
    output = pipe(prompt, max_new_tokens=350, temperature=0.1, do_sample=False)
    generated_text = output[0]['generated_text']

    # Clean the output to get just the model's response
    answer_only = generated_text.replace(prompt, "").strip()

    model_results.append({
        "Index": index,
        "Question": question,
        "Model's Logic": answer_only,
        "Target Answer": ground_truth
    })
    print(f"Done processing question {index+1}")

# 3. ANALYSIS & DISPLAY
results_df = pd.DataFrame(model_results)

# Number normalization function for accuracy check
def check_accuracy(row):
    def to_eng(text):
        p_digits = "۰۱۲۳۴۵۶۷۸۹"
        for i, d in enumerate(p_digits):
            text = text.replace(d, str(i))
        return text

    # We extract all numbers from the model output and see if the target is there
    model_nums = re.findall(r'\d+', to_eng(row["Model's Logic"]))
    target_num = to_eng(row["Target Answer"])
    return target_num in model_nums

results_df['Correct'] = results_df.apply(check_accuracy, axis=1)

# Display table
display(HTML("<h2>GSM8K Persian Evaluation Results</h2>"))
display(results_df.style.set_properties(**{
    'text-align': 'right',
    'direction': 'rtl',
    'white-space': 'pre-wrap'
}))

accuracy = results_df['Correct'].mean() * 100
print(f"\n⭐ Sample Accuracy: {accuracy}%")

--- Loading Model: arefehRajabian/qwenMath ---


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading adapter weights from arefehRajabian/qwenMath led to unexpected keys not found in the model: base_model.model.model.layers.0.mlp.down_proj.lora_A.default.weight, base_model.model.model.layers.0.mlp.down_proj.lora_B.default.weight, base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight, base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight, base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight, base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weight, base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight, base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight, base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight, base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight, base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight, base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight, base_model.model.model.layers.0.self_attn.v_proj.lora_A.de

--- Evaluating 10 GSM8K Questions ---
Done processing question 1
Done processing question 2
Done processing question 3
Done processing question 4
Done processing question 5
Done processing question 6
Done processing question 7
Done processing question 8
Done processing question 9
Done processing question 10


,Index,Question,Model's Logic,Target Answer,Correct
0,0,مرغ‌های جانت هر روز ۱۶ تخم می‌گذارند. او هر صبح سه تخم برای صبحانه می‌خورد و هر روز چهار تخم را برای پختن مافین برای دوستانش استفاده می‌کند. او باقی‌مانده تخم‌ها را به قیمت ۲ دلار برای هر تخم مرغ تازه در بازار کشاورزان به صورت روزانه می‌فروشد. او هر روز چقدر دلار در بازار کشاورزان به دست می‌آورد؟,خوب، ما باید مرحله به مرحله به این سوال پاسخ دهیم: 1. مرغ‌های جانت هر روز ۱۶ تخم می‌گذارند. 2. او سه تخم روزانه برای صبحانه می‌خورد: ۱۶ - ۳ = ۱۳ تخم باقی مانده 3. او هر روز چهار تخم را برای پختن مافین استفاده می‌کند: ۱۳ - ۴ = ۹ تخم باقی مانده 4. او هر روز ۹ تخم را به قیمت ۲ دلار برای هر تخم مرغ تازه در بازار کشاورزان فروش می‌دهد. 5. پس سوال آخر این است که چقدر دلار در بازار کشاورزان به دست می‌آورد؟ برای پاسخ به این سوال، باید تعداد تخم‌های باقی‌مانده (۹) را در قیمت هر تخم (۲ دلار) ضرب کنیم: 9 × 2 = 18 دلار پاسخ نهایی: جانت هر روز ۱۸ دلار در بازار کشاورزان به دست می‌آورد.,18,True
1,1,یک ردای عبا ۲ گُل (بُلت) الیاف آبی و نصف آن الیاف سفید نیاز دارد. در مجموع چند گُل الیاف لازم است؟,برای حل این مسئله، باید مرحله به مرحله فرآیندهای ریاضی را انجام دهیم: 1. تعریف متغیر: فرض کنیم اندازه ردای عبا را با x نشان دهیم. 2. معادله اول: یک ردای عبا 2 گُل الیاف آبی نیاز دارد. بنابراین: 2 = عدد گُل الیاف آبی 3. معادله دوم: نصف اندازه ردای عبا نیاز به گُل الیاف سفید دارد. بنابراین: x/2 = عدد گُل الیاف سفید 4. معادله سوم: مجموع گُل الیاف آبی و سفید برابر اندازه ردای عبا است. بنابراین: 2 + x/2 = x 5. حل معادله: 2 + x/2 = x 2 + x/2 - x/2 = x - x/2 2 = x - x/2 2 = x/2 2 * 2 = x 4 = x 6. پاسخ: اندازه ردای عبا 4 است. 7. محاسبه گُل الیاف: گُل الیاف,3,True
2,2,"جاش تصمیم می‌گیرد یک خانه را بازسازی و بفروشد. او یک خانه به قیمت ۸۰,۰۰۰ دلار می‌خرد و سپس ۵۰,۰۰۰ دلار برای تعمیرات آن هزینه می‌کند. این کار ارزش خانه را ۱۵۰ درصد افزایش می‌دهد. او چقدر سود کرد؟","خوب، ما مراحل زیر را طی می‌کنیم تا پاسخ را پیدا کنیم: 1. اولین مرحله: قیمت خرید خانه خانه را به قیمت 80,000 دلار می‌خرد. 2. دومین مرحله: هزینه تعمیرات 50,000 دلار برای تعمیرات هزینه می‌کند. 3. سومین مرحله: ارزش خانه بعد از تعمیرات ارزش خانه 150% افزایش می‌یابد. ارزش افزایش = 150% × (80,000 + 50,000) = 1.5 × 130,000 = 195,000 دلار 4. چهارمین مرحله: محاسبه سود سود = ارزش خانه بعد از تعمیرات - هزینه کل = 195,000 - (80,000 + 50,000) = 195,000 - 130,000 = 65,000 دلار پاسخ نهایی: جاش 65,000",70000,False
3,3,جیمز تصمیم می‌گیرد ۳ دوی سرعت را ۳ بار در هفته بدود. او در هر دوی سرعت ۶۰ متر می‌دود. او در مجموع چند متر در هفته می‌دود؟,خوب، ما مراحل زیر را برای حل این مسئله طی می‌کنیم: 1. جیمز ۳ دوی سرعت را ۳ بار در هفته بدود. 2. در هر دوی سرعت ۶۰ متر می‌دود. 3. ما باید مسافت کل را محاسبه کنیم که جیمز در هفته می‌دهد. برای این کار، ما باید: - مسافت یک دوی سرعت را در نظر بگیریم - آن را در تعداد دوی‌های سرعت در هفته ضرب کنیم 4. مسافت یک دوی سرعت: ۶۰ متر 5. تعداد دوی‌های سرعت در هفته: ۳ بار 6. مسافت کل در هفته = مسافت یک دوی سرعت × تعداد دوی‌های سرعت در هفته 7. مسافت کل در هفته = ۶۰ متر × ۳ = ۱۸۰ متر پاسخ نهایی: جیمز در هفته ۱۸۰ متر می‌دود.,540,False
4,4,هر روز، وندی به هر یک از مرغ‌هایش سه پیمانه خوراک مخلوط مرغ می‌دهد که شامل دانه‌ها، کرم خاکی و سبزیجات است تا به حفظ سلامتی آن‌ها کمک کند. او خوراک مرغ‌ها را در سه وعده جداگانه می‌دهد. صبح، ۱۵ پیمانه خوراک به گله مرغ‌هایش می‌دهد. بعدازظهر، ۲۵ پیمانه خوراک دیگر به مرغ‌هایش می‌دهد. اگر تعداد مرغ‌های وندی ۲۰ عدد باشد، او برای وعده نهایی روز به چند پیمانه خوراک نیاز دارد؟,برای حل این مسئله، باید مرحله به مرحله فرآیندهای ریاضی را انجام دهیم: 1. **جمع کل مقدار خوراکی که وندی به مرغ‌ها داده است:** - صبح: 15 پیمانه - بعدازظهر: 25 پیمانه جمع کل: \[ 15 + 25 = 40 \text{ پیمانه} \] 2. **تعداد مرغ‌ها:** - وندی 20 مرغ دارد. 3. **مقدار خوراکی که هر مرغ در روز می‌خورد:** - کل خوراک (40 پیمانه) بر روی 20 مرغ تقسیم می‌شود. \[ \frac{40}{20} = 2 \text{ پیمانه بر مرغ} \] 4. **مقدار خوراکی که هر مرغ در وعده نهایی می‌خورد:** - هر مرغ 2 پیمانه در روز می‌خورد. - وعده نهایی تنها شامل یک مرغ است. 5. **مقدار خوراکی که وعده نهایی می‌شود:** - هر مرغ 2,20,True
5,5,کیلار برای آپارتمان جدیدش به مغازه رفت تا لیوان بخرد. قیمت هر لیوان ۵ دلار است، اما هر لیوا


⭐ Sample Accuracy: 40.0%


In [19]:
import torch
import gc

# 1. Delete the previous model and pipeline
if 'pipe' in locals(): del pipe
if 'model' in locals(): del model
if 'tokenizer' in locals(): del tokenizer

# 2. Force garbage collection
gc.collect()

# 3. Clear the VRAM cache
torch.cuda.empty_cache()

print("GPU Memory Cleared! You can now load the next model.")

GPU Memory Cleared! You can now load the next model.


In [ ]:
import pandas as pd
import re
import torch
import gc
import time
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from IPython.display import display, HTML

# 1. SETUP MODEL (Re-initializing the pipe)
target_model = "arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit"

print(f"--- Loading Model: {target_model} ---")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(target_model)
model = AutoModelForCausalLM.from_pretrained(
    target_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Initialize the missing 'pipe'
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 2. CONFIGURATION & DATASET
# Using the translated_question column from your Kaggle dataset
eval_df = test_dataset.head(10)
system_prompt = " شما یک متخصص ریاضی هستید. سوال را مرحله به مرحله حل کنید و در انتها پاسخ نهایی را بنویسید از زیاده گویی پرهیز کن."

model_results = []

print(f"--- Evaluating {len(eval_df)} GSM8K Questions ---")

for index, row in eval_df.iterrows():
    question = row['translated_question']
    ground_truth = str(row['answer'])

    # Prompt structure
    prompt = f"{system_prompt}\n\nسوال: {question}\nپاسخ:"

    # Generate
    output = pipe(prompt, max_new_tokens=350, temperature=0.1, do_sample=False)
    generated_text = output[0]['generated_text']

    # Clean the output to get just the model's response
    answer_only = generated_text.replace(prompt, "").strip()

    model_results.append({
        "Index": index,
        "Question": question,
        "Model's Logic": answer_only,
        "Target Answer": ground_truth
    })
    print(f"Done processing question {index+1}")

# 3. ANALYSIS & DISPLAY
results_df = pd.DataFrame(model_results)

# Number normalization function for accuracy check
def check_accuracy(row):
    def to_eng(text):
        p_digits = "۰۱۲۳۴۵۶۷۸۹"
        for i, d in enumerate(p_digits):
            text = text.replace(d, str(i))
        return text

    # We extract all numbers from the model output and see if the target is there
    model_nums = re.findall(r'\d+', to_eng(row["Model's Logic"]))
    target_num = to_eng(row["Target Answer"])
    return target_num in model_nums

results_df['Correct'] = results_df.apply(check_accuracy, axis=1)

# Display table
display(HTML("<h2>GSM8K Persian Evaluation Results</h2>"))
display(results_df.style.set_properties(**{
    'text-align': 'right',
    'direction': 'rtl',
    'white-space': 'pre-wrap'
}))

accuracy = results_df['Correct'].mean() * 100
print(f"\n⭐ Sample Accuracy: {accuracy}%")

--- Loading Model: arefehRajabian/qwen2.5-3b-unsloth-bnb-4bit ---


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Device set to use cuda:0


--- Evaluating 10 GSM8K Questions ---
Done processing question 1
Done processing question 2
Done processing question 3
Done processing question 4
Done processing question 5


In [18]:
# This saves the results to a CSV file in your Colab sidebar
results_df.to_csv("qwen2.5-3b-unsloth-bnb-4bit-results.csv", index=False, encoding='utf-8-sig')

print("Results saved to gsm8k_evaluation_results.csv. You can download it from the folder icon on the left!")

Results saved to gsm8k_evaluation_results.csv. You can download it from the folder icon on the left!
